In [1]:
import numpy as np
import pandas as pd

import os, urllib.request, json , sys, tqdm

import matplotlib.pyplot as plt
plt.style.use('default')

sys.path.append('/'.join(os.getcwd().split('/')[:-1] + ['code']))
import risk_assessment as ra

In [2]:
symbols = [
#     'GOOGL',
    'X010.DE'
#     'AAPL'
]

In [3]:
time_deltas = [pd.to_timedelta(int(years),unit='Y') for years in np.arange(5,10,1)]

In [4]:
histograms_over_time = ra.calculate_risk_histogram_as_function_of_time(
    symbols[0],
    time_deltas
)

  0%|          | 0/9 [00:00<?, ?it/s]/Users/ingmar/projects/risk_assessment/code/risk_assessment.py:208: RuntimeWarning: divide by zero encountered in true_divide
  relative_growth = (pairwise_values[:,1] - pairwise_values[:,0])/pairwise_values[:,0]
/Users/ingmar/projects/risk_assessment/code/risk_assessment.py:208: RuntimeWarning: invalid value encountered in true_divide
  relative_growth = (pairwise_values[:,1] - pairwise_values[:,0])/pairwise_values[:,0]
/Users/ingmar/projects/risk_assessment/code/risk_assessment.py:212: RuntimeWarning: invalid value encountered in log
  np.log(relative_growth) / num_years
100%|██████████| 9/9 [2:11:09<00:00, 655.01s/it]   


In [7]:
bins_max

inf

In [8]:
bins_min

-0.9996018950997703

In [6]:
n_bins = 100

plt.figure(figsize=(16,9))
bins_min = min([np.nanmin(hist) for hist in histograms_over_time])
bins_max = max([np.nanmax(hist) for hist in histograms_over_time])

bins        = np.linspace(bins_min,bins_max,num=n_bins)
bin_centers = (bins[1:] + bins[:-1])/2

binned_over_time = np.zeros((len(bin_centers),len(time_deltas)))
first_quartiles = []
last_quartiles  = []

for ind, [timediff, histogram] in enumerate(zip(time_deltas,histograms_over_time)):
    
#     The distribution and the total number of samples differs for each time_delta,
#     therefore the binning is not done by the default routine but is fixed.
#     This allows to compare statistic relevance by the absolute values of the bin counts
#     that decreases for longer time differences (longer time diff means less samples
#     in fixed time span for which historic data is available)
    
#     drop nan
    histogram = histogram[~np.isnan(histogram)]
    
#     create histogram
    [hist_now,_] = np.histogram(histogram,bins)
    binned_over_time[:,ind] = hist_now
                   
#     calculate quartiles
    first_quartiles.append(
        np.percentile(histogram,25)
    )
    last_quartiles.append(
        np.percentile(histogram,75)
    )

    
time_deltas_years = [time_delta.total_seconds()/365/24/60/60 for time_delta in time_deltas]

plt.imshow(
    binned_over_time,
    aspect = 'auto',
    origin='lower',
    extent = [
        time_deltas_years[0],
        time_deltas_years[-1],
        bins_min,
        bins_max
    ]
)

plt.xticks(
    time_deltas_years,
    ["{:.2f}".format(time_delta) for time_delta in time_deltas_years]
)

bin_centers_subsample_indices = np.array([int(element) for element in np.array([0,0.25,.5,.75,1]) * (n_bins-2)])
bin_centers_subsample = bin_centers[bin_centers_subsample_indices]

# plt.yticks(
#     np.arange(len(bin_centers))[bin_centers_subsample_indices],
#     ["{:.1f}".format(bin_center) for bin_center in bin_centers_subsample]
# )

plt.title(symbols[0])
plt.ylabel('Effective annual growth rate')
plt.xlabel('Investment time in years')

plt.plot(time_deltas_years,first_quartiles,'red')
plt.plot(time_deltas_years,last_quartiles,'orange')
plt.plot(time_deltas_years,[0] * len(time_deltas_years),'--',)

plt.legend([
    'First Quartile',
    'Last Quartile'
])

plt.savefig('../output/figs/' + symbols[0] + '.pdf', dpi = 300)

/Users/ingmar/.anaconda/anaconda3/lib/python3.6/site-packages/numpy/core/function_base.py:133: RuntimeWarning: invalid value encountered in multiply
  y *= step


ValueError: range parameter must be finite.

<Figure size 1152x648 with 0 Axes>